In [1]:
import pandas as pd
from sklearn.cluster import AgglomerativeClustering as AC

data = pd.read_csv("Downloads/Crimes_-_2001_to_Present.csv");
data = data[:100000]
def scale_df(df,axis=0):
    return (df - df.mean(axis=axis)) / df.std(axis=axis)

def plot_hmap(df, ix=None, cmap='PuRd'):
    if ix is None:
        ix = np.arange(df.shape[0])
    plt.imshow(df.iloc[ix,:], cmap=cmap)
    plt.colorbar(fraction=0.03)
    plt.yticks(np.arange(df.shape[0]), df.index[ix])
    plt.xticks(np.arange(df.shape[1]))
    plt.grid(False)
    plt.show()
    

C:\Users\Mugdha\AppData\Local\Temp/ipykernel_25224/3799575682.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return (df - df.mean(axis=axis)) / df.std(axis=axis)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
def scale_and_plot(df, ix = None):
    df_marginal_scaled = scale_df(df.T).T
    if ix is None:
        ix = AC(4).fit(df_marginal_scaled).labels_.argsort()
    cap = np.min([np.max(df_marginal_scaled.as_matrix()), np.abs(np.min(df_marginal_scaled.as_matrix()))])
    df_marginal_scaled = np.clip(df_marginal_scaled, -1*cap, cap)
    plot_hmap(df_marginal_scaled, ix=ix)

scale_and_plot(data)

In [3]:
data.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,...,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"


In [6]:
for crime in data['Primary Type']:
    print(crime.lower())
    break

battery


In [10]:
import gensim

# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('Downloads/GoogleNews-vectors-negative300.bin', binary=True)  

In [13]:
model.vector_size

300

In [14]:
crime_type_vectors = []
for crime in data['Primary Type']:
    crime = crime.lower()
    if crime in model:
        crime_type_vectors.append(model[crime])

In [15]:
crime_type_vectors[0]

array([-0.01708984,  0.27539062,  0.35742188, -0.48046875, -0.03710938,
        0.15917969,  0.01239014, -0.01647949,  0.25390625, -0.14941406,
       -0.16210938,  0.16601562, -0.38085938,  0.15625   ,  0.09521484,
        0.07666016,  0.19628906, -0.16894531,  0.03100586, -0.11572266,
        0.10742188, -0.2734375 , -0.30664062, -0.11865234,  0.02062988,
        0.12988281, -0.14257812,  0.35742188,  0.18652344,  0.18066406,
        0.00823975,  0.13085938, -0.4609375 ,  0.08886719, -0.265625  ,
       -0.13867188,  0.13183594,  0.21777344, -0.09863281,  0.41796875,
       -0.10546875,  0.33398438, -0.11767578,  0.21386719, -0.08349609,
       -0.14257812,  0.25585938,  0.15917969,  0.06201172, -0.27734375,
       -0.13085938, -0.16796875, -0.35742188,  0.26367188,  0.01953125,
        0.11865234, -0.0703125 , -0.20996094,  0.06542969,  0.12695312,
       -0.06738281,  0.13574219, -0.1953125 ,  0.09472656,  0.09716797,
        0.01586914, -0.43359375, -0.10009766, -0.15527344,  0.20

In [16]:
from sklearn.cluster import KMeans

In [28]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(crime_type_vectors, data['IUCR'])

In [29]:
kmeans.labels_

array([1, 0, 0, ..., 0, 0, 1])

In [30]:
kmeans.labels_[:10]

array([1, 0, 0, 2, 3, 4, 4, 0, 4, 0])

In [22]:
import matplotlib.pyplot as plt
import numpy as np

TypeError: '(array([False,  True,  True, ...,  True,  True, False]), 0)' is an invalid key